In [ ]:
!pip install transformers==2.10.0

In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer


model_name_or_path = "sberbank-ai/rugpt3large_based_on_gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name_or_path)
model = GPT2LMHeadModel.from_pretrained(model_name_or_path).cuda()



model.eval()
model.to('cuda')


In [ ]:
!pip install pyTelegramBotApi

In [4]:
import telebot

In [12]:
def generate(text,stop=18,limit=True,temp=0.15,max=40,k=0,p=0.95,rp=1.0,num=1,no_repeat_ngram_size=2,num_beams=1,early_stopping=False,length_penalty=1):
    input_ids = tokenizer.encode(text, return_tensors='pt')
    input_ids = input_ids.to('cuda')
    greedy_output = model.generate(input_ids, max_length=len(input_ids[0])+max, temperature=temp, num_beams=num_beams,early_stopping=early_stopping, top_k=k, top_p=p, no_repeat_ngram_size=no_repeat_ngram_size, num_return_sequences=num,do_sample=True, repetition_penalty=rp,eos_token_id=stop,length_penalty=length_penalty)
    if num > 1:
        outs = []
        for out in greedy_output:
            outs.append(tokenizer.decode(out, skip_special_tokens=True))
        return outs
    return tokenizer.decode(greedy_output[0], skip_special_tokens=True)

In [27]:
prompt = """Bot-wiki"""

history = {}

def get_answer(question,id):
  question = question.capitalize()
  if not question[-1] in ["?","!","."]:
    question = question + "."
  gen = generate(prompt.format(question))
  ans = gen.replace(prompt.format(question),"")
  print(gen)
  if ans[0] == " ":
    ans = ans[1:]
  return ans

In [34]:
import re
def clean_generated(prompt,gen):
    gen = gen.replace(prompt,'')
    tmp = gen.split('\n')
    find = re.findall(r'(.*?\?)', tmp[0])
    if len(find) > 1:
        tmp[0] = find[0]
    gen = tmp[0]
    prompt = prompt + gen
    return prompt,gen
def clean_before(prompt):
    while str.find(prompt," .") != -1:
        prompt = prompt.replace(" .",'.')
    while str.find(prompt," !") != -1:
        prompt = prompt.replace(" !",'!')
    while str.find(prompt," ?") != -1:
        prompt = prompt.replace(" ?",'?')
    while str.find(prompt," ,") != -1:
        prompt = prompt.replace(" ,",',')
    return prompt
class Brain:
    def __init__(self,name,prompt="",memory=[]):
        self.name = name
        self.prompt = prompt
        self.memory = memory
    def say_to_brain(self,who,text):
        ttmp = list(text)
        ttmp[0] = ttmp[0].capitalize()
        if not ttmp[-1] in '.?!,':
            ttmp.append(".")

        text = "".join(ttmp)

        self.memory.append("{}: {}".format(who,text))

        textmem = "\n\n".join(self.memory)

        textmem += "\n\n{}: ".format(self.name)
        memtok = tokenizer.encode( self.prompt + textmem, return_tensors='pt' )
        if len(memtok[0]) > 1900:
            tmpmem = textmem
            tmp = tokenizer.encode( self.prompt + tmpmem, return_tensors='pt' )
            while len(tmp[0]) > 1900:
                tmpmem = tmpmem[100:]
                tmp = tokenizer.encode( self.prompt + tmpmem, return_tensors='pt' )
            textmem = tmpmem
            
        textmem = clean_before(textmem)
        gen = generate(self.prompt + textmem,num_beams=1,early_stopping=True,no_repeat_ngram_size=2,temp=0.5,k=40,p=0.92)

        p,g = clean_generated(self.prompt + textmem, gen)
        self.memory.append("{}: {}".format(self.name,g))
        return g

In [45]:
bot = telebot.TeleBot(token="6006315509:AAF3PWQgVq35OgA1H1Ob9qiFCu6InBKU4f0")

bots = {}

@bot.message_handler(commands=['start'])
def start(message):
  global bots
  bots[message.from_user.id] = Brain("Википедия","Википедия - чат-бот. Википедия - виртуальный собеседник. Википедия - виртуальный цифровой помощник.\n\n")
  bot.send_message(message.chat.id, "Какие у тебя есть вопросы ?")

@bot.message_handler(content_types=['text'])
def text(message):
  global bots
  if not message.from_user.id in bots:
    bots[message.from_user.id] = Brain("Википедия","Википедия - чат-бот. Википедия - виртуальный собеседник. Википедия - виртуальный цифровой помощник.\n\n")
  ans = bots[message.chat.id].say_to_brain("Человек",message.text)
  print(f"Q: {message.text}\nA: {ans}")
  bot.send_message(message.chat.id, ans)

bot.polling(none_stop=True)

Q: 123
A: Задачей программиста является написание программного обеспечения под конкретную задачу пользователя путём создания программной реализации алгоритма работы программы средствами языка программирования высокого уровня абстракции ЛИНЕЙНОГО ИНТЕЛЛЕК
